# 맛집 위치를 지도에 표시하기

In [22]:
import folium
import pandas as pd
import googlemaps
import numpy as np
from tqdm import tqdm

In [23]:
df = pd.read_csv('03. best_sandwitches_list_chicago2.csv', index_col ='Rank')
df.head()

,Name,Menu,price,address,phone,website
Rank,,,,,,
1,BLT,Old Oak Tap,10,2109 W Chicago Ave,773-772-0406,theoldoaktap.com
2,Fried Bologna,Au Cheval,9,800 W Randolph St,312-929-4580,aucheval.tumblr.com
3,Woodland Mushroom,Xoco,9,50 445 N Clark St,312-334-3688,rickbayless.com
4,Roast Beef,Al’s Deli,9,40 914 Noyes St,847-475-9400,alsdeli.net
5,PB&L,Publican Quality Meats,10,825 W Fulton Mkt,312-445-8977,publicanqualitymeats.com


In [24]:
# 파일을 열어보기?
key_fd = open('googlemapskey.txt',mode='r')
gmaps_key = key_fd.read(100)
key_fd.close()
gmaps = googlemaps.Client(key = gmaps_key)

In [25]:
geo = gmaps.geocode('Delaware', language = 'en')
geo[0].get('geometry')

{'bounds': {'northeast': {'lat': 39.8394839, 'lng': -74.98416499999999},
  'southwest': {'lat': 38.451018, 'lng': -75.78914790000002}},
 'location': {'lat': 38.9108325, 'lng': -75.52766989999999},
 'location_type': 'APPROXIMATE',
 'viewport': {'northeast': {'lat': 39.8394839, 'lng': -74.98416499999999},
  'southwest': {'lat': 38.451018, 'lng': -75.78914790000002}}}

In [26]:
lat = []
lng = []

for address in tqdm(df['address']):
    if 'Multiple' not in address:
        target_name = address + ', Chicago'
        gmaps_geocode = gmaps.geocode(target_name, language='en')
        
        location_output = gmaps_geocode[0].get('geometry')
        lat.append(location_output['location']['lat'])
        lng.append(location_output['location']['lng'])
    
    else:
        lat.append(np.nan)
        lng.append(np.nan)


100%|██████████| 50/50 [00:05<00:00,  8.42it/s]


In [27]:
lat[:10], lng[:10]

([41.8956049,
  41.8846582,
  41.8905226,
  42.0583217,
  41.8866036,
  41.9002461,
  41.8590541,
  41.9102031,
  41.9097558,
  41.8345302],
 [-87.67996149999999,
  -87.6476668,
  -87.6307834,
  -87.6837479,
  -87.64853649999999,
  -87.62516339999999,
  -87.6252007,
  -87.68287529999999,
  -87.7176727,
  -87.6456492])

- Pandas의 df['열이름'].mean() 메소드를 사용하기 위해. 자동적으로 nan값을 제외하고 return한다

In [28]:
df['Latitude'] = lat
df['Longitude'] = lng
df.head(10)

,Name,Menu,price,address,phone,website,Latitude,Longitude
Rank,,,,,,,,
1,BLT,Old Oak Tap,10,2109 W Chicago Ave,773-772-0406,theoldoaktap.com,41.895605,-87.679961
2,Fried Bologna,Au Cheval,9,800 W Randolph St,312-929-4580,aucheval.tumblr.com,41.884658,-87.647667
3,Woodland Mushroom,Xoco,9,50 445 N Clark St,312-334-3688,rickbayless.com,41.890523,-87.630783
4,Roast Beef,Al’s Deli,9,40 914 Noyes St,847-475-9400,alsdeli.net,42.058322,-87.683748
5,PB&L,Publican Quality Meats,10,825 W Fulton Mkt,312-445-8977,publicanqualitymeats.com,41.886604,-87.648536
6,Belgian Chicken Curry Salad,Hendrickx Belgian Bread Crafter,7,25 100 E Walton St,312-649-6717,NaN,41.900246,-87.625163
7,Lobster Roll,Acadia,16,1639 S Wabash Ave,312-360-9500,acadiachicago.com,41.859054,-87.625201
8,Smoked Salmon Salad,Birchwood Kitchen,10,2211 W North Ave,773-276-2100,birchwoodkitchen.com,41.910203,-87.682875
9,Atomica Cemitas,Cemitas Puebla,9,3619 W North Ave,773-772-8435,cemitaspuebla.com,41.909756,-87.717673


In [30]:
''' 50개 위치의 평균값 두기 '''
# 지도 위치 잡기
mapping = folium.Map([df['Latitude'].mean(), df['Longitude'].mean()], zoom_start = 11)
# 지도 위에 마커 표시하기
folium.Marker([df['Latitude'].mean(), df['Longitude'].mean()], popup = 'center').add_to(mapping)

mapping

In [38]:
mapping = folium.Map( [df['Latitude'].mean(), df['Longitude'].mean()] , zoom_start = 11)

for index in df.index:
    if 'Multiple' not in df['address'][index]:
        folium.Marker( [ df['Latitude'][index], df['Longitude'][index] ] ,popup = df['Name'][index] ).add_to(mapping)

mapping